## **Programa de Pós-Graduação em Computação - INF/UFRGS**
### Disciplina CMP263 - Aprendizagem de Máquina
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br>

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. *


---

<br>

## **Tópico: Avaliação de Modelos Preditivos**

<br>

Até o momento trabalhamos com a avaliação de modelos preditivos utilizando a estratégia mais simples, de **Holdout** - uma simples divisão aleatória dos dados em conjuntos de treinamento e teste (Holdout de 2 vias) ou em conjuntos de treinamento, validação e teste (Holdout de 3 vias). No entanto, um treinamento e avaliação robustos de modelos preditivos demanda o uso de outras estratégias de divisão de dados, especialmente quando estamos trabalhando com um conjunto de dados de tamanho limitado. Dentre estas estratégias, destacamos o *k-fold cross-validation*.

O *k-fold cross-validation* (ou validação cruzada k-fold) e suas variantes (como *leave-one-out cross-validation* e *nested cross-validation*) estabeleceram-se como métodos referência para avaliar modelos em Aprendizado de Máquina pois mitigam diversas limitações relacionadas ao uso de Holdout, como i) permitir avaliar o desempenho de modelos preditivos com variações de conjuntos de dados de treinamento/teste (e assim evitar que os resultados dependam de uma escolha aleatória particular de divisão de dados), e ii) permitir que toda instância no conjunto de dados seja usada uma vez para avaliação do modelo.

<br>

**Objetivo deste notebook**: Explorar o uso de k-fold cross-validation para treinamento e avaliação de modelos preditivos, e otimização de hiperparâmetros.
<br>

---



##**Diagnóstico de Câncer de Mama**

Nesta atividade, vamos analisar o conjunto de dados relacionado ao diagnóstico de câncer de mama. Cada instância se refere ao exame de um(a) paciente. Os atributos são computados a partir de uma imagem digitalizada de material coletado de uma massa mamária através de uma punção aspirativa por agulha fina (PAAF). Por intermédio deste procedimento, é possível obter células de uma suspeita de lesão, que usualmente são analisadas com o auxílio de um microscópio pelo médico patologista. Os dados a serem utilizados definem um conjunto de atributos que descrevem as características dos núcleos celulares presentes na imagem, com o intuito de automatizar o processo de análise e definição do diagnóstico provável.

Dez características foram analisadas para cada núcleo celular:

*   raio (média das distâncias do centro aos pontos do perímetro)
*   textura (desvio padrão dos valores de escala de cinza)
*   perímetro
*   área
*   suavidade (variação local nos comprimentos dos raios)
*   compacidade (perímetro^2 / área - 1,0)
*   concavidade (gravidade das porções côncavas do contorno)
*   pontos côncavos (número de porções côncavas do contorno)
*   simetria
*   dimensão fractal

Para cada característica foram extraídas a média, o erro padrão e o pior (ou maior) valor, resultando em 30 atributos para cada exame. A última coluna, 'diagnosis', contém a classe verdadeira de cada instância, que pode ser M (maligno) ou B (benigno).





---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline
import pandas as pd             # biblioteca para análise de dados
import matplotlib.pyplot as plt # biblioteca para visualização de informações
import seaborn as sns           # biblioteca para visualização de informações
import numpy as np              # biblioteca para operações com arrays multidimensionais
sns.set()

## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Bibliotecas para converter variáveis categóricas (strings) para numéricas
from sklearn.preprocessing import OrdinalEncoder

sns.set()


In [ ]:
## Carregando os dados
data = pd.read_table("https://drive.google.com/uc?export=view&id=1S-qqiA7cISZzRsLBmBoXyVYO5opY4WEa")
data.head()  # para visualizar apenas as 5 primeiras linhas


A coluna *'diagnosis'* contém a classificação de cada amostra referente ao tipo de tumor, se maligno (M) ou benigno (B). Vamos avaliar como as instâncias estão distribuídas entre as classes presentes no dataset.

In [ ]:
## Distribuição do atributo alvo, 'diagnosis'
plt.hist(data['diagnosis'])
plt.title("Distribuição do atributo alvo - diagnosis")
plt.show()


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar os valores categóricos em inteiros a fim de ampliar as opções de algoritmos que podemos utilizar no treinamento dos modelos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = data.iloc[:, :-1].values
y = data.iloc[:, 30].values

## substitui 'B' por 0, 'M' por 1
y = np.array([0 if y=='B' else 1 for y in y])

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42)

In [ ]:
## Faz a normalização dos dados.
## Para fins de uso ao longo de todo o notebook, faremos
## separadamento entre treino e teste.
## Conforme discutiremos ao longo do curso,
## o conceito de 'Pipelines' pode ser usado para aprimorar essa solução.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler .transform(X_test)

---


### Treinamento de modelos com k-fold cross-validation

O scikit-learn possui amplo suporte para uso de k-fold cross-validation. Existem duas funções no scikit-learn que você pode usar para realizar a validação cruzada, a função [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) e a função [`cross_validate`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html). A função `cross_val_score` está no scikit-learn há muito tempo e tem uma interface muito simples, enquanto a função `cross_validate` foi adicionada posteriormente, é um pouco mais poderosa e oferece mais opções (como especificar múltiplas métricas para avaliação). No entanto, ambas têm uma interface muito semelhante e são fáceis de usar. Uma questão importante é que elas permitem especificar tanto o algoritmo de aprendizado (estimator) como detalhes da divisão de dados (número de folds, [estratégia de divisão](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators), etc.) Por padrão, ambas usam 5-fold CV.

O K-fold cross-validation (K-fold CV) visa de certa forma 'substituir' uma simples divisão entre treino/validação para o desenvolvimento dos modelos. Assim, vamos aplicar o k-Fold CV na partição `X_train` usando uma árvore de decisão e um KNN (por enquanto, sem otimização de hiperparâmetros). Preparamos o procedimento de k-Fold CV antes da função a fim de gerar a mesma partição de treino/teste para cada fold no caso de comparar múltiplos algoritmos. Usamos a versão `StratifiedKFold` para gerar uma divisão estratificada em relação às classes (para regressão, ou para problemas bem balanceados, podemos usar `KFold`)

#### Usando a função cross_val_score

In [ ]:
cv_5f = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

## Avalia uma árvore de decisão com 5-fold CV
clf_dt = DecisionTreeClassifier(max_depth=5,class_weight='balanced',random_state=42)
scores_dt = cross_val_score(estimator=clf_dt, X=X_train, y=y_train,scoring='recall',cv=cv_5f)

## Avalia um 5-NN com 5-fold CV
clf_knn = KNeighborsClassifier(n_neighbors=5)
scores_knn = cross_val_score(estimator=clf_knn, X=X_train, y=y_train,scoring='recall',cv=cv_5f)

In [ ]:
print(scores_dt)
print(np.mean(scores_dt))
print(np.std(scores_dt))

In [ ]:
print(scores_knn)
print(np.mean(scores_knn))
print(np.std(scores_knn))

In [ ]:
results=[]
results.append(scores_dt)
results.append(scores_knn)
plt.boxplot(results, tick_labels=['DT','KNN'], showmeans=True)
plt.show()

A função [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) tem uma interface similar a `cross_val_score`, mas retorna a predição de cada instância quando a mesma foi alocada ao conjunto de teste.

In [ ]:
predict_dt = cross_val_predict(estimator=clf_dt, X=X_train, y=y_train,cv=cv_5f)#,method='predict_proba') ##descomente para retornar as probabilidades preditas
print(predict_dt)

#### Usando a função cross_validate

A função permite estabelecer um conjunto de métricas de avaliação no parâmetro [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

Vamos utilizar para fins de exemplo, Recall, Precisão e Acurácia.


In [ ]:
scoring = ['recall','precision','accuracy']

In [ ]:
## Avalia uma árvore de decisão com 5-fold CV
scores_dt2 = cross_validate(estimator=clf_dt, X=X_train, y=y_train,scoring=scoring,cv=cv_5f)

## Avalia um 5-NN com 5-fold CV
scores_knn2 = cross_validate(estimator=clf_knn, X=X_train, y=y_train,scoring=scoring,cv=cv_5f)

In [ ]:
scores_dt2_df = pd.DataFrame(scores_dt2, columns=scores_dt2.keys())
scores_dt2_df

In [ ]:
plt.boxplot(scores_dt2_df[["test_recall","test_precision","test_accuracy"]],tick_labels=['Recall','Precision','Accuracy'],showmeans=True)
plt.show()

In [ ]:
scores_knn2_df = pd.DataFrame(scores_knn2, columns=scores_dt2.keys())
scores_knn2_df

In [ ]:
plt.boxplot(scores_knn2_df[["test_recall","test_precision","test_accuracy"]],tick_labels=['Recall','Precision','Accuracy'],showmeans=True)
plt.show()

#### Usando o Repeated KFold
Abaixo vamos realizar o treinamento dos dois modelos, baseado em árvores de decisão e baseado em KNN, utilizando o RepeatedKFold. O processo é exatamente igual ao que foi feito anteriormente, a diferença é que agora obtemos 15 estimativas de desempenho para cada modelo.

In [ ]:
repcv_5f = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)

## Avalia uma árvore de decisão com 5-fold CV repetido 3 vezes
scores_dt_rep = cross_validate(estimator=clf_dt, X=X_train, y=y_train,scoring=scoring,cv=repcv_5f)

## Avalia um 5-NN  com 5-fold CV repetido 3 vezes
scores_knn_rep = cross_validate(estimator=clf_knn, X=X_train, y=y_train,scoring=scoring,cv=repcv_5f)

In [ ]:
scores_dt2rep_df = pd.DataFrame(scores_dt_rep, columns=scores_dt_rep.keys())
scores_dt2rep_df

In [ ]:
plt.boxplot(scores_dt2rep_df[["test_recall","test_precision","test_accuracy"]],tick_labels=['Recall','Precision','Accuracy'],showmeans=True)
plt.show()

In [ ]:
scores_knn2rep_df = pd.DataFrame(scores_knn_rep, columns=scores_knn_rep.keys())
scores_knn2rep_df

In [ ]:
plt.boxplot(scores_knn2rep_df[["test_recall","test_precision","test_accuracy"]],tick_labels=['Recall','Precision','Accuracy'],showmeans=True)
plt.show()

Pelo critério de Recall, o modelo de Árvores de decisão obteve os melhores resultados, com mediana e média mais altas. Por outro lado, o KNN possui melhor precisão e uma maior acurácia média.

A escolha do melhor modelo não é direta ou trivial. No domínio médico, a sensibilidade (recall) é uma métrica muito relevante. Assim, o modelo de  Árvores de decisão poderia ser escolhido dentre os dois analisados  para treinar um modelo a partir de todos os dados de treinamento, e avaliar seu desempenho final nos dados de teste. Este desempenho final é a estimativa do poder preditivo que poderíamos obter caso este modelo seja aplicado para auxiliar no diagnóstico precoce de diabetes.

In [ ]:
#from sklearn.metrics import roc_curve, auc
dt_final = clf_dt.fit(X_train,y_train)
y_predProba_dt = dt_final.predict_proba(X_test)
y_pred_dt = dt_final.predict(X_test)

print(metrics.recall_score(y_test,y_pred_dt))
print(metrics.precision_score(y_test,y_pred_dt))
print(metrics.accuracy_score(y_test,y_pred_dt))

In [ ]:
y_predProba_dt

---

### Otimização de hiperparâmetros com GridSearch e Nested Cross-validation

O sklearn disponibiliza a função `GridSearchCV`, que permite realizar a otimização de hiperparâmetros de forma prática usando k-fold cross-validation. A chamada da função é muito simples:



```
search = GridSearchCV(estimator, param_grid, scoring='recall', n_jobs=1, cv=5, refit=True)
```

A função, conforme chamada acima, utiliza um modelo (`estimator`) para explorar as combinações de valores de hiperparâmetros definidos em `param_grid,` através de um 5-fold cross-validation (determinado por `cv`) e escolhe o melhor modelo a partir da métrica de Recall(também configurável). A opção `refit=True` define que após determinar a melhor configuração de hiperparâmetros, a mesma será utilizada para retreinar um modelo com todos os dados utilizados no processo de GridSearch com CV.



A validação cruzada aninhada (nested cross-validation) é frequentemente usada para treinar um modelo no qual os hiperparâmetros também precisam ser otimizados. A seleção de modelo sem CV aninhado usa os mesmos dados para ajustar os hiperparâmetros do modelo e avaliar o desempenho do modelo. As informações podem, portanto, “vazar” para o modelo, que acaba se sobreajustando aos dados. O resultado deste vazamento pode ser uma avaliação excessivamente otimista. A magnitude desse efeito depende principalmente do tamanho do conjunto de dados e da estabilidade do modelo.

Na célula abaixo, vamos executar o k-fold cross-validation e o nested cross-validation, treinando uma árvore de decisão com pré-poda. O hiperparâmetro a ser otimizado é o `max_depth`. Para os resultados mostrados abaixo, o desempenho refere-se sempre ao melhor valor de hiperparâmetro obtido a cada iteração do GridSearch.

In [ ]:
#Adaptado de: https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html

from sklearn.model_selection import GridSearchCV

# Número de execuções da validação cruzada (aninhada/não aninhada)
NUM_TRIALS = 30

# Grid de hiperparâmetros para otimização
param_grid = {"max_depth": [1,2,3,4,5,6,7]}

# Uso de uma árvore de decisão como algoritmo de aprendizado
model = DecisionTreeClassifier()


# Arrays para armazenar os scores de cada abordagem
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop para múltiplas execuções, cada qual com um random_state distinto
for i in range(NUM_TRIALS):

    # Definir estratégias de validação cruzada para o loop interno (inner) e para
    # o loop externo (outer). As escolhas podem ser diferentes. Aqui, optamos por
    # usar StratifiedKFold em ambas.
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)

    # GridSearch e avaliação dos modelos na abordagem de k-fold cross-validation padrão
    # (sem aninhamento) - avalia os modelos com os mesmos dados usados para selecionar hiperparâmetros
    clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=outer_cv) ## sem aninhamento - desempenho com melhores hiperparâmetros
    clf.fit(X_train, y_train)
    non_nested_scores[i] = clf.best_score_

    # GridSearch e avaliação dos modelos na abordagem de nested k-fold cross-validation
    clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, refit=True) ##inner CV - melhores hiperparâmetros
    nested_score = cross_val_score(clf, X=X_train, y=y_train, cv=outer_cv) ##outer CV - desempenho com melhores hiperparâmetros (do inner)
    nested_scores[i] = nested_score.mean()

## Calcula a diferença de scores entre as abordagens
score_difference = non_nested_scores - nested_scores

print(
    "Average difference of {:6f} with std. dev. of {:6f}.".format(
        score_difference.mean(), score_difference.std()
    )
)

# Plotar um gráfico com os scores obtidos e com a diferença entre eles
# em cada iteração
plt.figure(figsize=(15,15))
plt.subplot(211)
(non_nested_scores_line,) = plt.plot(non_nested_scores, color="r")
(nested_line,) = plt.plot(nested_scores, color="b")
plt.ylabel("score", fontsize="14")
plt.legend(
    [non_nested_scores_line, nested_line],
    ["Non-Nested CV", "Nested CV"],
    bbox_to_anchor=(0, 0.4, 0.5, 0),
)
plt.title(
    "Non-Nested and Nested Cross Validation on Breast Cancer Dataset",
    x=0.5,
    y=1.1,
    fontsize="15",
)

# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend(
    [difference_plot],
    ["Non-Nested CV - Nested CV Score"],
    bbox_to_anchor=(0, 1, 0.8, 0),
)
plt.ylabel("score difference", fontsize="14")

plt.show()

A célula a seguir faz a otimização de hiperparâmetros com nested cross-validation para o algoritmo de árvore de decisão.

In [ ]:
# define o modelo
model = DecisionTreeClassifier()
# define o espaço de busca de hiperparâmetros
param_grid = dict()
param_grid['max_depth'] = [3,5,7,None]
param_grid['min_samples_split'] = [5,15,30,None]

### loop interno ####
# configura o loop interno do nested cross-validation
cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# define a estratégia de busca dos melhores hiperparâmetros (baseado em recall)
search = GridSearchCV(model, param_grid, scoring='recall', n_jobs=1, cv=cv_inner, refit=True)
### loop interno ####

### loop externo ####
# configura o loop externo do nested cross-validatiion
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# executa o nested cross-validation
output_ncv = cross_validate(search, X_train, y_train, scoring=scoring, cv=cv_outer, n_jobs=-1,return_estimator=True, return_train_score=True)

# reporta os resultados
pd.DataFrame(output_ncv)

Podemos avaliar os resultados do processo de nested cross-validation observando a variação de desempenho ao longo das múltiplas execuções e os melhores valores de hiperparâmetros selecionados em cada iteração. Cabe salientar que esta análise é feita com base no número de repetições do outer cross-validation (neste caso, configurado com 10 folds)

In [ ]:
# Análise de recall
mean_val_score = output_ncv['test_recall'].mean()

print('nested_train_scores: ', output_ncv['train_recall'])
print('nested_val_scores:   ', output_ncv['test_recall'])
print('mean score:            {0:.2f}'.format(mean_val_score))

In [ ]:
# Análise de precisão
mean_val_score = output_ncv['test_precision'].mean()

print('nested_train_scores: ', output_ncv['train_precision'])
print('nested_val_scores:   ', output_ncv['test_precision'])
print('mean score:            {0:.2f}'.format(mean_val_score))

In [ ]:
## Quais os melhores valores de hiperparâmetros de acordo com o nested Cross_validation?
[x.best_params_ for x in output_ncv['estimator']]

O conhecimento a respeito dos melhores hiperparâmetros ao longo do nested cross-validation pode guiar nossa decisão sobre que modelos (algoritmos e configuração de hiperparâmetros) podemos utilizar para gerar o modelo final. Este modelo final seria obtido treinando um modelo com estas configurações sobre todo o conjunto de dados usado no processo de avaliação.

